In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
#pra receber os dadoss

try:
    print("Carregando dados...")
    ENDERECO_DADOS = 'https://www.ispdados.rj.gov.br/Arquivos/BaseDPEvolucaoMensalCisp.csv'
    df = pd.read_csv(ENDERECO_DADOS, sep=';', encoding='iso-8859-1') #  pra corrigir os dados
    # determmina as colunas relevantes e converter para datetime
    df['mes_ano'] = pd.to_datetime(df['ano'].astype(str) + '-' + df['mes'].astype(str))
    #junta por trimestre
    df_trimestral = df.groupby(pd.PeriodIndex(df['mes_ano'], freq='Q'))[['lesao_corp_dolosa', 'lesao_corp_morte']].sum().reset_index()
    print('\nDados trimestrais:')
    print(df_trimestral.head())
    print('\nEstatisticas descritivas:')
    print(df_trimestral.describe())
except Exception as e:
    print("Erro ao carregar os dados:", {e})
    exit()


Carregando dados...

Dados trimestrais:
  mes_ano  lesao_corp_dolosa  lesao_corp_morte
0  2003Q1              20963                15
1  2003Q2              18585                19
2  2003Q3              17991                16
3  2003Q4              20689                 6
4  2004Q1              19470                10

Estatisticas descritivas:
       lesao_corp_dolosa  lesao_corp_morte
count          90.000000         90.000000
mean        18003.266667         11.355556
std          3122.183336          4.460340
min          8307.000000          4.000000
25%         15875.500000          8.000000
50%         18151.500000         11.000000
75%         20254.500000         14.000000
max         25500.000000         26.000000


In [2]:
 #2. Análise de Correlação
try:
    print('\nCalculando correlação...')
    
    correlacao = np.corrcoef(df_trimestral['lesoes_dolosas'], 
                            df_trimestral['lesoes_morte'])[0,1]
    
    print(f'Correlação Pearson entre lesoes dolosas e com morte: {correlacao:.3f}')
    
    # Gráfico de dispersão
    plt.figure(figsize=(10,6))
    plt.scatter(df_trimestral['lesoes_dolosas'], 
                df_trimestral['lesoes_morte'],
                alpha=0.7)
    plt.title(f'Relação entre Lesoes Dolosas e com Morte (Correlação: {correlacao:.3f})')
    plt.xlabel('Lesoes Dolosas (3 meses)')
    plt.ylabel('Lesoes com Morte (3 meses)')
    plt.grid(True)
    plt.show()
    
except Exception as e:
    print(f'Erro na análise de correlação: {e}')
    exit()


Calculando correlação...
Erro na análise de correlação: 'lesoes_dolosas'


: 